In [34]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import cv2
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from keras.callbacks import LearningRateScheduler
from keras.layers import Dropout
from keras.callbacks import Callback
from tensorflow.keras import layers, models
import tensorflow as tf
# 图片路径
image_folder = "E:\shujuji\metamaterialpicture90"
# Excel文件路径
excel_path = "E:\\1-1000-90.xlsx"
df = pd.read_excel(excel_path, usecols=[0, 16])  # 使用第1列和第17列的数据，从0开始计数，并跳过第一行
# 初始化图像和性能数组
X = []
y = []
for i in range(1, 651):
    img_path = os.path.join(image_folder, f"lcy{i}.jpg")
    if i in df["编号"].values:
        img = load_img(img_path, target_size=(224, 224))  # 调整为合适的图像大小
        img_array = img_to_array(img)
        img_array = cv2.cvtColor(img_array.astype('uint8'), cv2.COLOR_RGB2GRAY)
        X.append(img_array)
y=df["d33"]
# 转换为NumPy数组
X = np.array(X)
y = np.array(y)
# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=500)
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)
X_train = X_train.astype('float32')
y_train = y_train.astype('float32')
X_test = X_test.astype('float32')
y_test = y_test.astype('float32')
print(X_train.shape)

(510, 224, 224, 1)


In [35]:
# 定义ResNet的基本块
# 定义Residual Block
# 定义Residual Block
def residual_block(x, filters, kernel_size=3, stride=1):
    # 主分支
    y = layers.Conv2D(filters, kernel_size=kernel_size, strides=stride, padding='same')(x)
    y = layers.BatchNormalization()(y)
    y = layers.Activation('relu')(y)

    y = layers.Conv2D(filters, kernel_size=kernel_size, strides=1, padding='same')(y)
    y = layers.BatchNormalization()(y)

    # 调整形状以匹配
    if stride > 1:
        x = layers.Conv2D(filters, kernel_size=1, strides=stride, padding='same')(x)
        x = layers.BatchNormalization()(x)

    # 跳跃连接
    out = layers.add([x, y])
    out = layers.Activation('relu')(out)
    return out

# 定义ResNet-18模型
def build_resnet(input_shape):
    input_tensor = layers.Input(shape=input_shape)
    
    x = layers.Conv2D(64, (7, 7), strides=(2, 2), padding='same')(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    x = residual_block(x, 64)
    x = residual_block(x, 64)
    x = residual_block(x, 128, stride=2)
    x = residual_block(x, 128)
    x = residual_block(x, 256, stride=2)
    x = residual_block(x, 256)
    x = residual_block(x, 512, stride=2)
    x = residual_block(x, 512)

    x = layers.GlobalAveragePooling2D()(x)
    
    output_tensor = layers.Dense(1, activation='linear')(x)

    model = models.Model(inputs=input_tensor, outputs=output_tensor)
    return model


# 构建ResNet模型
input_shape = X_train.shape[1:]
resnet_model = build_resnet(input_shape)

# 编译模型
resnet_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [ ]:
# 训练模型4
history =resnet_model.fit(X_train, y_train, epochs=30, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/30
8/8 [==============================] - 30s 3s/step - loss: 199844.4844 - mae: 438.9090 - val_loss: 1517872640.0000 - val_mae: 38948.1445
Epoch 2/30
8/8 [==============================] - 25s 3s/step - loss: 190871.5781 - mae: 428.7538 - val_loss: 4836093952.0000 - val_mae: 69507.2344
Epoch 3/30
8/8 [==============================] - 26s 3s/step - loss: 186188.5938 - mae: 423.4979 - val_loss: 1414637952.0000 - val_mae: 37588.4141
Epoch 4/30
8/8 [==============================] - 26s 3s/step - loss: 181791.8125 - mae: 418.5195 - val_loss: 209737760.0000 - val_mae: 14474.1504
Epoch 5/30
8/8 [==============================] - 26s 3s/step - loss: 177323.8594 - mae: 413.4179 - val_loss: 45647268.0000 - val_mae: 6752.9834
Epoch 6/30
8/8 [==============================] - 26s 3s/step - loss: 172699.3906 - mae: 408.1357 - val_loss: 16673230.0000 - val_mae: 4080.7273
Epoch 7/30
8/8 [==============================] - 27s 3s/step - loss: 167966.2656 - mae: 402.7691 - val_loss: 843406.75

In [ ]:
plt.plot(history.history['loss'][5:], label='Training Loss')
plt.plot(history.history['val_loss'][5:], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend() 
plt.show()

In [ ]:
# 在训练集上计算 R² 分数
y_train_pred = resnet_model.predict(X_train)
r2_train = r2_score(y_train, y_train_pred)

# 在验证集上计算 R² 分数
y_test_pred = resnet_model.predict(X_test)
r2_test = r2_score(y_test, y_test_pred)

# 打印 R² 分数
print(f'R² 分数 - 训练集: {r2_train:.4f}')
print(f'R² 分数 - 验证集: {r2_test:.4f}')